## Temporary notebook for tuning of CNMF source extraction parameters
In this notebook, a single-plane MMAP file is specified as input. The MMAP file should be concatenated across trials and motion corrected.

## Imports

In [1]:
import os, sys, time, copy
import numpy as np
import matplotlib.pyplot as plt
import ipyparallel

%matplotlib inline

In [2]:
import caiman as cm
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params

In [3]:
os.environ['MKL_NUM_THREADS']='1'
os.environ['OPENBLAS_NUM_THREADS']='1'

### Input file

In [ ]:
data_folder = '/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/parameterTuning'
# data_folder = '/home/luetcke/test/M5.2/20181211/S1/parameterTuning'
# mmap_file = '20181211_S1_Join_G0_F707_P0_rig_remFrames_d1_84_d2_508_d3_1_order_C_frames_704_.mmap'
frame_rate = 10.2

In [ ]:
# this cell is tagged as `parameters`
mmap_file = '20181211_S1_Join_G0_F707_P0_rig_remFrames_d1_84_d2_508_d3_1_order_C_frames_704_.mmap'

In [ ]:
mmap_file = os.path.join(data_folder, mmap_file)

print('\nUsing input file %s\n' % (mmap_file))

### Compute params

In [ ]:
ncpus = 8

### Setup cluster

In [ ]:
%%bash -s "$ncpus"
source /opt/Anaconda3-5.1.0-Linux-x86_64/bin/activate caiman || source activate caiman
ipcluster stop
sleep 5
ipcluster start --daemonize -n $1

In [ ]:
time.sleep(5)
# connect client
client = ipyparallel.Client()
time.sleep(2)
while len(client) < ncpus:
    sys.stdout.write(".")  # Give some visual feedback of things starting
    sys.stdout.flush()     # (de-buffered)
    time.sleep(0.5)

# create dview object
client.direct_view().execute('__a=1', block=True)
dview = client[:]
n_processes = len(client)
print('\n\nThe cluster appears to be setup. Number of parallel processes: %d' % (n_processes))

### Parameters for source extraction
Define the important parameters for calcium source extraction.

In [ ]:
# dataset dependent parameters
decay_time = 0.4                            # length of a typical transient in seconds

# parameters for source extraction and deconvolution
p = 1                         # order of the autoregressive system
gnb = 2                       # number of global background components
merge_thresh = 0.8            # merging threshold, max correlation allowed
rf = [7,14]                   # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
rf = None
stride_cnmf = 3               # amount of overlap between the patches in pixels
K = 20                        # max. number of components per patch
gSig = [7,35]                 # expected half size of neurons in pixels

method_init = 'sparse_nmf'    # initialization method (if analyzing dendritic data use 'sparse_nmf', else 'greedy_roi')
#alpha_snmf = 10e2            # sparsity penalty for dendritic data analysis through sparse NMF
alpha_snmf = 1
normalize_init = True         # default is True
sigma_smooth_snmf = (0.5, 1, 1) # defaults to (0.5, 0.5, 0.5)
max_iter_snmf = 500           # defaults to 500

ssub = 1                      # spatial subsampling during initialization
tsub = 1                      # temporal subsampling during intialization

In [ ]:
# create Parameters object
# unspecified parameters get default values
opts_dict = {'fnames': [mmap_file],
             'fr': frame_rate,
            'decay_time': decay_time,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K,
             'gSig': gSig,
            'stride': stride_cnmf,
            'method_init': method_init,
            'alpha_snmf': alpha_snmf,
            'normalize_init': normalize_init,
            'sigma_smooth_snmf': sigma_smooth_snmf,
            'max_iter_snmf': max_iter_snmf,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub}

opts = params.CNMFParams(params_dict=opts_dict)

### Prepare data for source extraction

In [ ]:
Yr, dims, T = cm.load_memmap(mmap_file)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

### Display frame average

In [ ]:
plt.figure(figsize=(30,30))
avg_img = np.mean(images,axis=0)
plt.imshow(avg_img, cmap='gray'), plt.title('Frame average', fontsize=32);

In [ ]:
frame_ix = 100
plt.figure(figsize=(30,30))
plt.imshow(images[frame_ix,:,:], cmap='gray'), plt.title('Frame %d' % (frame_ix), fontsize=32);

### Run CNMF

In [ ]:
t_start = time.time()

# Extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0)
opts.set('temporal', {'p': 0})
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm.fit(images)
     
# Re-run seeded CNMF on accepted patches to refine and perform deconvolution
cnm.params.set('temporal', {'p': p})
cnm2 = cnm.refit(images, dview=dview)

t_elapsed = time.time() - t_start
print('\nFinished Source Extract in %1.2f s' % (t_elapsed))

### Evaluate components

In [ ]:
cnm2.dview = None
cnm_final = copy.deepcopy(cnm2)

In [ ]:
# Parameters for evaluation
quality_params = {
    'min_SNR': 3,               # signal to noise ratio for accepting a component
    'rval_thr': 0.99,           # space correlation threshold for accepting a component
    'use_cnn': False,           # use CNN classifier
    'cnn_thr': 0.95,            # threshold for CNN based classifier
    'cnn_lowest': 0.1           # neurons with cnn probability lower than this value are rejected
}

opts.set('quality', quality_params)
cnm_final.estimates.evaluate_components(images, opts, dview=dview)
print('Found %d good / %d bad components\n' % (len(cnm_final.estimates.idx_components), 
                                               len(cnm_final.estimates.idx_components_bad)))

In [ ]:
# calculate correlation image
cc = cm.local_correlations(images.transpose(1,2,0))
cc[np.isnan(cc)] = 0

In [ ]:
plt.imshow(cc)

In [ ]:
def nb_view_components(cnm_list, img, good_or_bad='good'):
    '''
    View components in Caiman slider plot for different planes. 
    Choose 'good' or 'bad' to display accepted or rejected components.
    '''
    for ix_plane, cnm in enumerate(cnm_list):
        show_plot = True
        print('Plane %d' % (ix_plane))
        if good_or_bad == 'good':
            component_list = cnm.estimates.idx_components
        elif good_or_bad == 'bad':
            component_list = cnm.estimates.idx_components_bad
        
        if len(component_list) == 0:
            print('No valid %s components in this plane!\n\n' % (good_or_bad))
            show_plot = False
        elif len(component_list) == 1: # adress caiman bug if only 1 component
            print('Found 1 %s component. Duplicating due to Caiman bug.' % (good_or_bad))
            component_list = np.append(component_list, component_list[0])
        if show_plot:
            cnm.estimates.nb_view_components(img=img, idx=component_list)

### Interactive component plot

In [ ]:
# accepted components
# nb_view_components([cnm_final], cc, good_or_bad='good')

In [ ]:
# rejected components
# nb_view_components([cnm_final], cc, good_or_bad='bad')

In [ ]:
def plot_component_contours(cnm, images, good_or_bad, data_folder, file_stem):
    avg_img = np.mean(images,axis=0)
    if good_or_bad == 'good':
        idx_comps = cnm.estimates.idx_components
    elif good_or_bad == 'bad':
        idx_comps = cnm.estimates.idx_components_bad
    
    A = cnm.estimates.A
    A_dense = A.todense()

    counter = 1
    plt.figure(figsize=(30,30));
    for i_comp in range(len(idx_comps)):
        plt.subplot(len(idx_comps),2,counter)
        if counter == 1:
            plt.title('CNMF Components', fontsize=24);

        counter += 1
        dummy = cm.utils.visualization.plot_contours(A[:,idx_comps[i_comp]], avg_img, cmap='gray', 
                                                     colors='r', display_numbers=False)
        component_img = np.array(np.reshape(A_dense[:,idx_comps[i_comp]], avg_img.shape, order='F'))
        plt.subplot(len(idx_comps),2,counter)
        counter += 1
        plt.imshow(component_img), plt.title('Component %1.0f' % (i_comp), fontsize=24)

    plt.tight_layout()

    out_file = '%sComponentContours_%s_test.png' % (file_stem, good_or_bad)
    fig_name = os.path.join(data_folder, out_file)

    plt.savefig(fig_name)
    plt.close()

In [ ]:
file_stem = os.path.basename(mmap_file).replace('.mmap','')

In [ ]:
plot_component_contours(cnm_final, images, 'good', data_folder, file_stem)

In [ ]:
plot_component_contours(cnm_final, images, 'bad', data_folder, file_stem)

### Stop cluster

In [ ]:
%%bash
source /opt/Anaconda3-5.1.0-Linux-x86_64/bin/activate caiman || source activate caiman
ipcluster stop
sleep 1